In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L4\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['Capital.csv', 'Central.csv', 'Dunwoody.csv', 'Genese.csv', 'Hudson.csv', 'LongIsland.csv', 'Milwood.csv', 'NYC.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[1]))

,Demand,Normalized_net,Temprature
0,1813.7,0.610002,-5.8
1,1737.4,0.584340,-6.1
2,1700.7,0.571997,-7.5
3,1683.3,0.566145,-8.4
4,1688.4,0.567860,-7.4
...,...,...,...
43819,1947.5,0.655003,3.4
43820,1881.5,0.632805,3.3
43821,1804.6,0.606941,2.6
43822,1734.2,0.583264,2.7


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[1]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temprature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_Central,Temp_Central
0,1813.7,-5.8
1,1737.4,-6.1
2,1700.7,-7.5
3,1683.3,-8.4
4,1688.4,-7.4
...,...,...
43819,1947.5,3.4
43820,1881.5,3.3
43821,1804.6,2.6
43822,1734.2,2.7


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_Central,Temp_Central,Lag-2,Lag-7
0,2113.2,-13.9,1935.8,1813.7
1,2062.7,-14.2,1897.8,1737.4
2,2041.6,-14.8,1869.6,1700.7
3,2040.6,-15.6,1868.5,1683.3
4,2097.5,-16.0,1913.6,1688.4
...,...,...,...,...
43651,1947.5,3.4,1979.2,1846.2
43652,1881.5,3.3,1920.9,1826.5
43653,1804.6,2.6,1821.3,1805.3
43654,1734.2,2.7,1723.8,1738.8


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 17s 11ms/step - loss: 0.0144 - mape: 6399.5620 - mae: 0.0949 - val_loss: 0.0098 - val_mape: 114.4603 - val_mae: 0.0814
Epoch 2/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0096 - mape: 6344.8936 - mae: 0.0769 - val_loss: 0.0090 - val_mape: 108.9015 - val_mae: 0.0760
Epoch 3/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0071 - mape: 7132.0688 - mae: 0.0657 - val_loss: 0.0089 - val_mape: 81.6624 - val_mae: 0.0727
Epoch 4/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0061 - mape: 6761.8096 - mae: 0.0602 - val_loss: 0.0081 - val_mape: 92.7753 - val_mae: 0.0709
Epoch 5/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0055 - mape: 5182.4160 - mae: 0.0570 - val_loss: 0.0075 - val_mape: 47.4575 - val_mae: 0.0654
Epoch 6/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0051 - mape: 4959.3574 - mae: 0.0546 - val

1105/1105 [==============================] - 12s 11ms/step - loss: 0.0024 - mape: 2053.4500 - mae: 0.0373 - val_loss: 0.0075 - val_mape: 89.9974 - val_mae: 0.0634
Epoch 48/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0024 - mape: 1709.6918 - mae: 0.0372 - val_loss: 0.0084 - val_mape: 114.5407 - val_mae: 0.0644
Epoch 49/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0023 - mape: 2000.0237 - mae: 0.0368 - val_loss: 0.0081 - val_mape: 102.8075 - val_mae: 0.0655
Epoch 50/100
1105/1105 [==============================] - 13s 11ms/step - loss: 0.0023 - mape: 1593.0070 - mae: 0.0364 - val_loss: 0.0081 - val_mape: 108.9515 - val_mae: 0.0652
Epoch 51/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0023 - mape: 2104.6738 - mae: 0.0363 - val_loss: 0.0087 - val_mape: 114.4304 - val_mae: 0.0670
Epoch 52/100
1105/1105 [==============================] - 13s 11ms/step - loss: 0.0022 - mape: 1480.8213 - mae: 0.0360 - val_loss

1105/1105 [==============================] - 12s 11ms/step - loss: 0.0012 - mape: 1132.5953 - mae: 0.0270 - val_loss: 0.0102 - val_mape: 123.9152 - val_mae: 0.0723
Epoch 94/100
1105/1105 [==============================] - 13s 11ms/step - loss: 0.0013 - mape: 767.7687 - mae: 0.0272 - val_loss: 0.0095 - val_mape: 119.8883 - val_mae: 0.0703
Epoch 95/100
1105/1105 [==============================] - 13s 11ms/step - loss: 0.0012 - mape: 943.5735 - mae: 0.0263 - val_loss: 0.0104 - val_mape: 126.4151 - val_mae: 0.0740
Epoch 96/100
1105/1105 [==============================] - 13s 11ms/step - loss: 0.0012 - mape: 971.9782 - mae: 0.0264 - val_loss: 0.0100 - val_mape: 125.6964 - val_mae: 0.0716
Epoch 97/100
1105/1105 [==============================] - 13s 11ms/step - loss: 0.0012 - mape: 1247.1857 - mae: 0.0262 - val_loss: 0.0107 - val_mape: 121.8816 - val_mae: 0.0745
Epoch 98/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0012 - mape: 1206.0841 - mae: 0.0265 - val_loss: 

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1252.3907544612885


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 7ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.09004185, 0.06655023, 0.06084581, ..., 0.4211714 , 0.36518553,
       0.32583106], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.19491731, 0.1775025 , 0.16371577, ..., 0.34768765, 0.30839269,
       0.27685624])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,1530.9,1343.007446
1,1499.7,1300.920410
2,1475.0,1290.700439
3,1486.7,1308.134888
4,1534.1,1345.334961
...,...,...
4363,1947.5,2118.111084
4364,1881.5,2047.588501
4365,1804.6,1936.251953
4366,1734.2,1835.948730


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

7.004683517796793